In [1]:
import os
import requests
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
# from openai import AzureOpenAI

# from langchain.chat_models import init_chat_model

from langchain.tools import tool
from langchain.agents import create_agent

pwd = %pwd
env_path = os.path.join(pwd,'.env')

load_dotenv(env_path)


/mnt/custom-file-systems/efs/fs-0713103aada1ad94d_fsap-03d52ef0d5b91b6d3/LLM-World/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.getenv('AZURE_OPENAI_BASEURL')

In [ ]:
# os.environ

In [ ]:
# response = requests.get(os.getenv('AZURE_OPENAI_BASEURL'))
# response.json()['nonprod'].keys()
# response.json()['nonprod']['dall-e-3']


In [ ]:
response = requests.get(os.getenv('AZURE_OPENAI_BASEURL'))

model_endpoint = {}
for obj_name in response.json()['nonprod']:
    model_endpoint[response.json()['nonprod'][obj_name][0]['deployment_name']]=response.json()['nonprod'][obj_name][0]['endpoint']

model_endpoint.keys()

In [ ]:
model_endpoint['gpt-4.1']

In [ ]:
modle_id = 'gpt-4.1'
model = AzureChatOpenAI(
    deployment_name=modle_id,  
    openai_api_version="2025-03-01-preview",
    # openai_api_version="2024-12-01-preview",
    azure_endpoint=model_endpoint['gpt-4.1'],
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    temperature=0)
    


In [ ]:
model

In [ ]:
@tool
def create_calendar_event(
    title:str,
    start_time:str,
    end_time:str,
    atendees:list[str],
    location:str=""
) -> str:
    """create calendar event"""
    return f"Created event: {title} from {start_time} to {end_time} with {len(atendees)} atendees."

@tool
def send_email(
    to:list[str],
    subject:str,
    body:str,
    cc:list[str]
) -> str:
    """send an email via api"""
    return f"Email sent to {','.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    atendees:list[str],
    date:str,
    duration_minutes:int,
)-> list[str]:
    """Check calendar availability for a given atendee on a specific date"""
    return ["09:00","14:00","16:00"]

### Subagent

In [ ]:
CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (eg, 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response. "
)



calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [ ]:
calendar_agent

In [ ]:
## test agent

query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

## surface real-time response from agent
for step in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    #stream_mode="updates" #emits an event after every step
):
    print(step.values())

In [ ]:
## test agent

query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

## surface real-time response from agent (progress, intermediate steps, partial responses as they are generated)
for step in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    #stream_mode="updates" #emits an event after every step
):
    # print(step.values())
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

In [ ]:
for chunk in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    stream_mode="updates" #emits an event after every step
):
    print(chunk.items())

In [ ]:
for chunk in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    stream_mode="updates" #emits an event after every step
):

    for step, data in chunk.items():
        print(f"steps: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

In [ ]:
for chunk in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    stream_mode="updates" #emits an event after every step
):

    for step, data in chunk.items():
        print(f"steps: {step}")
        print(f"content: {data['messages'][-1].content}")

In [ ]:
## final answer w/o realtime feedback
response = calendar_agent.invoke({"messages":[{"role":"user","content":query}]})
response

In [ ]:
# response['messages'][-1].content
response['messages'][-1].text

In [ ]:
## eamil agent

EMAIL_AGENT_PROMPT = (
    "You are an email assitant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft apropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response. "
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT
)

In [ ]:
query = "send the design team a reminder about reviewing the new mock"

for step in email_agent.stream({"messages": [{"role":"user", "content":query}]}):
    for update in step.values():
        for message in update.get("messages",[]):
            message.pretty_print()

In [ ]:
response = email_agent.invoke({"messages":[{"role":"user","content":query}]})
response

In [ ]:
response['messages'][-1].content
# response['messages'][-1].text

#### Wrap subagent as a tool for supervisor to call

In [ ]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.


    Input: Natural language scheduling request (e.g., 'meeting with design team next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages":[{"role":"user", "content":request}]
        })
    return result['messages'][-1].content


@tool
def manage_email(request: str) -> str:
    """ Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any
    communication. Handles recipient extraction, subject generation, and email composisiton.

    Input: Natural language email request (e.g., 'send time reminder about the meeting' )
    """
    result = email_agent.invoke({
        "messages":[{"role":"user", "content":request}]
    })
    return result['messages'][-1].text
    

##### Supervisor Agent

In [ ]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down uer requests into appropriate tool calls and coorddinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence. "
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT
)

In [ ]:
query = "Schedule a team standup for tomorrow at 9am"

for steps in supervisor_agent.stream(
    {"messages":[{"role":"user","content":query}]}):
    for update in steps.values():
        for messages in update.get("messages",[]):
            messages.pretty_print()